# Introduction

We need to first import the necessary packages and files:

In [1]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
from pathlib import Path
from numpy import exp
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [2]:
heart_disease_training= pd.read_csv("C:/Users/Calvi/Documents/Data Science Toolbox/Assessment 1/data/TrainingDataImputation.csv")
heart_disease_test = pd.read_csv("https://raw.githubusercontent.com/erinp0/DST-Assessment-1/main/Erin%20Pollard/test_imputed.csv")

We then separate the features and target variables:

In [3]:
y_train = np.array(heart_disease_training.iloc[:, 0]).astype(int)[np.newaxis,:].T
y_test = np.array(heart_disease_test.iloc[:, 0]).astype(int)[np.newaxis,:].T
del heart_disease_training['HeartDiseaseorAttack']
del heart_disease_test['HeartDiseaseorAttack']

In [4]:
x_train = heart_disease_training.to_numpy()
x_test = heart_disease_test.to_numpy()

In [5]:
def center_function(data):
    mean = data.mean(axis=1)
    std = data.std(axis=1)
    for i in range(data.shape[1]):
        data[i] -= mean[i]
        data[i] /= std[i]
    return data,mean,std

In [6]:
cf = center_function(x_train)
x_train_centered = cf[0]

In [7]:
print("Standardised Training Data:")
print(pd.DataFrame(x_train_centered).head())

Standardised Training Data:
         0         1         2         3         4         5         6   \
0  1.392049  0.099506  1.392049 -0.495064  0.099506  0.099506 -0.331341   
1  0.827005  0.827005  0.827005 -1.354394  0.827005 -0.731137 -1.250518   
2  1.013108  1.013108  1.013108 -1.047109  1.013108 -0.398000 -0.868369   
3 -0.099784  1.527512 -1.727079 -0.718156  1.527512 -0.099784 -0.642215   
4  0.693649  0.693649  0.693649 -1.398234  0.693649  0.693649 -1.243280   

         7         8         9   ...        11        12        13        14  \
0 -1.193037 -1.193037 -1.193037  ...  1.392049  0.099506  0.530354 -1.109647   
1 -0.731137  0.827005  0.827005  ...  0.827005 -0.731137  0.827005  0.827005   
2 -0.398000  1.013108  1.013108  ...  1.013108 -1.809108  0.072370 -1.718068   
3 -0.099784 -0.099784 -0.099784  ...  1.527512 -0.099784  0.442648 -1.622092   
4  0.693649  0.693649  0.693649  ...  0.693649  0.693649  0.693649 -2.211744   

         15        16        17        1

# PCA

In [8]:
percentage = 99 # This is the amount of explained variance we desire

The function below retrives and orders the eigenvectors and values of the covariance matrix of the data:

In [9]:
def cov_eigen(data):
    hd_scaled = StandardScaler().fit_transform(data)
    features = hd_scaled.T
    cov_matrix = np.cov(features)
    values, vectors = np.linalg.eig(cov_matrix)
    idx = values.argsort()[::-1]
    vectors = vectors.T[:,idx]
    return values,vectors

The pcafeatures function calculates the amount of components to have a certain amount of explained variance and outputs that,the explained variances and a list of the principle component names

In [10]:
def pcafeatures(data,percentage):
    hd_scaled = StandardScaler().fit_transform(data)
    features = hd_scaled.T
    cov_matrix = np.cov(features)
    values, vectors = np.linalg.eig(cov_matrix)
    explained_variances = []
    a = 0
    num = 0
    for i in range(len(values)):
        explained_variances.append(values[i] / np.sum(values))
    for i in range(len(explained_variances)):
        while a < percentage:
            a += explained_variances[i]*100
            num += 1
    pc_feat = ['PC{}'.format(i+1) for i in range(num)]
    return pc_feat,num,explained_variances

In [16]:
print("Checking explained variance:")
print(str(pcafeatures(x_train_centered,percentage)[2]))
a = np.sum(pcafeatures(x_train_centered,percentage)[2])
print("Sum of explained variance is",a)

Checking explained variance:
[0.1348172518376701, 0.13032279301032065, 0.0955261856838543, 0.05931948440589512, 0.05573780236659315, 0.050289472213804365, 0.04707748013447749, 0.04502433250643572, 0.04147180060574487, 0.03935159134585836, 0.018428583013388018, 0.01968953915778834, 0.0224107095867694, 0.023331563422669396, 0.02507554363585574, 0.02679305099499639, 0.036237484854748536, 0.034725178890560066, 0.03356557452555949, 0.030312543120407234, 0.030492034686603404]
Sum of explained variance is 1.0000000000000002


In [ ]:
num_of_comps = pcafeatures(x_train_centered,percentage)[1]
cov_evecs = cov_eigen(x_train_centered)[1][:,:num_of_comps]

In [ ]:
print("Covariance Eigenvectors:")
print(pd.DataFrame(cov_evecs).head())

In [ ]:
x_train_pca = np.dot(x_train_centered,cov_evecs)

In [ ]:
print(pd.DataFrame(x_train_pca[:5,:], columns = pcafeatures(x_train_centered,percentage)[0]))

# Logistic Regression

In [ ]:
#%% Parameter initialize and sigmoid function

def initialize_weights_and_bias(dimension):
    w = np.full((dimension,1),0.01)
    b = 0.0
    return w,b

def sigmoid(z):
    y_head = 1 / (1+exp(-z))
    return y_head

The sigmoid function is used for deciding the probability. It's clear from the graph below that the the higher the input it, the closer it tends to 1 and the lower it is, the closer it tends to 0. As we are working with binary classification, the only outcomes we desire are 1 or 0.

In [ ]:
x_axis = np.linspace(-10,10,20)
plt.plot(x_axis,sigmoid(x_axis))
plt.axhline(y=1,color = 'r',linestyle = '-')
plt.axhline(y=0,color = 'r',linestyle = '-')
plt.axhline(y=0.5,color = 'b', linestyle = ':')
plt.axvline(x=0,color = 'r', linestyle = ':')
plt.title("Sigmoid Function")

plt.show()

In [ ]:
#%% Loss function

def logloss(y_hat,y):
    a = np.log(y_hat)
    b = np.log(1-y_hat)
    loss = (-1/x_train.shape[0]) * np.sum(y*a + (1-y)*b)
    return loss

In [ ]:
#%% Forward and Backward Propagation

def forward_backward_propagation(w,b,x_train,y_hat):
    
    z = np.dot(x_train,w) + b
    y_hat = sigmoid(z)
    cost = logloss(y_hat,y_train)
    
    #backward propogation
    derivative_weight = np.dot((y_hat-y_train).T,x_train)/x_train.shape[0]
    derivative_bias = np.sum(y_hat-y_train)/x_train.shape[0]
    gradients = {"derivative_weight": derivative_weight,"derivative_bias": derivative_bias}
    return cost,gradients

In [ ]:
#%% Updating (Learning) Parameters
    
def update(w, b, x_train, y_train, learning_rate,number_of_iteration):
    cost_list = []
    cost_list2 = []
    index = []
    # updating(learning) parameters is number_of_iteration times
    for i in range(number_of_iteration):
        # make forward and backward propagation and find cost and gradients
        cost,gradients = forward_backward_propagation(w,b,x_train,y_train)
        cost_list.append(cost)
        # lets update
        w = w - learning_rate * (gradients["derivative_weight"].T)
        b = b - learning_rate * gradients["derivative_bias"]
        if i % 50 == 0:
            cost_list2.append(cost)
            index.append(i)
            print ("Cost after iteration %i: %f" %(i, cost)) #if section defined to print our cost values in every 10 iteration. We do not need to do that. It's optional.
     # we update(learn) parameters weights and bias
    parameters = {"weight": w,"bias": b}
    plt.plot(range(len(cost_list)),cost_list)
    plt.xticks(index,rotation='vertical')
    plt.xlabel("Number of Iteration")
    plt.ylabel("Cost")
    plt.show()
    return parameters, gradients, cost_list 

In [ ]:
#%% Prediction

def predict(w,b,x_test):
    # x_test is a input for forward propagation
    z = sigmoid(np.dot(x_test,w)+b)
    Y_prediction = np.zeros((x_test.shape[0],1))
    # if z is bigger than 0.5, our prediction is one means has diabete (y_head=1),
    # if z is smaller than 0.5, our prediction is zero means does not have diabete (y_head=0),
    for i in range(len(z)):
        if z[i]< 0.5:
            Y_prediction[i] = 0
        else:
            Y_prediction[i] = 1

    return Y_prediction

#predict(parameters["weight"],parameters["bias"],x_test)

In [ ]:
def logistic_regression(x_train, y_train, x_test, y_test, learning_rate ,  num_iterations):
    # initialise
    dimension =  x_train.shape[1]
    w,b = initialize_weights_and_bias(dimension)
    
    parameters, gradients, cost_list = update(w, b, x_train, y_train, learning_rate,num_iterations)
    
    y_prediction_test = predict(parameters["weight"],parameters["bias"],x_test)

    # Print train/test Errors
    
    print("test accuracy: {} %".format(100 - np.mean(np.abs(y_prediction_test - y_test)) * 100))

The probabilities funtion outputs the probabilities of each observable testing positive or negative

In [ ]:
def probabilities(x_train, x_test, y_train, learning_rate,number_of_iteration):
    # initialise
    dimension =  x_train.shape[1]
    w,b = initialize_weights_and_bias(dimension)
    # updating(learning) parameters is number_of_iteration times
    for i in range(number_of_iteration):
        # make forward and backward propagation and find cost and gradients
        cost,gradients = forward_backward_propagation(w,b,x_train,y_train)
        # lets update
        w = w - learning_rate * (gradients["derivative_weight"].T)
        b = b - learning_rate * gradients["derivative_bias"]
        z = sigmoid(np.dot(x_test,w)+b)
    return z

We need to prepare our test dataset before predicting the outcome. We need to project the test data into the same space as the training. So we follow the standardising method but use the mean and standard deviation from the training data

In [ ]:
for i in range(x_test.shape[1]):
    x_test[:,i] -= cf[1][i]     
    x_test[:,i] /= cf[2][i]

In [ ]:
x_test_pca = np.dot(x_test,cov_evecs)

# With and without PCA Comparison

## Without PCA:

In [ ]:
start = time.time()
logistic_regression(x_train, y_train, x_test, y_test,learning_rate = 2, num_iterations = 235)
end = time.time()

In [ ]:
print("Without PCA, execution time of Logistic Regression is: " + str(end-start))

## With PCA:

In [ ]:
start=time.time()
logistic_regression(x_train_pca, y_train, x_test_pca, y_test,learning_rate =4 , num_iterations = 246)
end=time.time()

In [ ]:
print("With PCA, execution time of Logistic Regression is: " + str(end-start))

We can look at the probabilities of a case being at risk of heart disease or an attack using our model:

In [ ]:
y_hat = probabilities(x_train_pca, x_test_pca, y_train, learning_rate = 4,number_of_iteration = 246)

In [ ]:
y_hat = pd.DataFrame(y_hat, columns = ['Probabilities'])

In [ ]:
print(y_hat)